# **APPLIED STATISTICS PROJECT**

## **Supervised Analysis**

## Useful libraries and packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import mudata
import scanpy as sc
import anndata as ad

import pingouin as pg

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report



## Discrimiant Analysis

In [ ]:
# Load datasets prepared in the preprocessing script
scores_df = pd.read_csv('/Users/eli/Desktop/progetto applied/dataset_applied/rna_scores_30pcs.csv', index_col=0)
loadings_df = pd.read_csv("/Users/eli/Desktop/progetto applied/dataset_applied/rna_loadings_30pcs.csv")
grouped_rna = pd.read_csv("/Users/eli/Desktop/progetto applied/dataset_applied/median_with_age.csv")

In [ ]:
# Check dimensions
print(scores_df.shape)
print(loadings_df.shape)

(434, 33)
(36601, 31)


In [ ]:
scores_df.head()

,patient_id,age,cell_type,PC1,PC2,PC3,PC4,PC5,PC6,PC7,...,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30
0,SRR16208932,Pediatric_AML,B cell,-11.941673,4.346546,0.998468,-2.427639,2.210926,2.572245,-1.495453,...,0.460158,0.692598,-0.066943,-0.115917,0.206061,-0.472407,0.796603,0.206891,-0.431710,-0.585791
1,SRR16208932,Pediatric_AML,B_prog,-5.553754,4.568302,-1.620732,-0.471580,-0.864127,2.427597,0.813819,...,-0.720072,-0.664382,1.463664,-2.573327,0.354794,-0.441758,0.812932,-0.538093,-0.313310,-0.478668
2,SRR16208932,Pediatric_AML,CLP,4.626499,3.222884,-1.639073,0.373265,-3.467237,2.125864,3.426461,...,4.014057,-15.627580,-13.765819,-5.509414,7.331663,7.134823,-4.235648,-1.053140,4.682926,1.055341
3,SRR16208932,Pediatric_AML,Early GMP,-1.759837,6.098621,-1.379452,2.790820,-3.916896,1.683125,0.765121,...,0.359933,0.741603,0.977658,-0.324565,0.517452,-0.976356,-0.503475,0.370270,1.100100,0.156464
4,SRR16208932,Pediatric_AML,Erythroid,-8.575211,2.042565,-3.060355,3.342749,0.942301,-1.462913,7.875761,...,-0.108303,0.343222,0.002711,-0.602981,0.852930,-0.527701,0.616021,0.110478,-0.471771,-0.008115


In [ ]:
# Display the columns and rename the first one
print(loadings_df.columns)

loadings_df = loadings_df.rename(columns={'Unnamed: 0': 'genes'})
print(loadings_df.columns)


Index(['Unnamed: 0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8',
       'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17',
       'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26',
       'PC27', 'PC28', 'PC29', 'PC30'],
      dtype='object')
Index(['genes', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9',
       'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18',
       'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27',
       'PC28', 'PC29', 'PC30'],
      dtype='object')


In [ ]:
loadings_df.head()

,genes,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30
0,MIR1302-2HG,-4.112838e-18,-5.568293e-17,-5.082637e-17,-1.407835e-16,-1.332556e-17,-8.414932e-17,-1.018236e-16,3.627997e-16,-2.437136e-17,...,5.286072e-17,5.408346e-17,2.799740e-17,7.706299e-17,8.451824e-17,5.849730e-17,-9.073039e-17,-6.801545e-17,1.619604e-16,-5.244800e-17
1,FAM138A,-6.331741e-17,4.336809e-17,-9.020562e-17,-1.597572e-16,4.943962e-17,9.671083e-17,-2.428613e-17,-3.469447e-18,6.938894e-18,...,-1.179612e-16,3.903128e-18,7.632783e-17,-5.551115e-17,1.040834e-17,5.903481e-17,-9.925871e-17,-4.857226e-17,3.989864e-17,-6.245005e-17
2,OR4F5,-4.465181e-17,6.150701e-17,-7.694165e-17,-4.027674e-17,-1.019827e-16,-5.135450e-18,-5.687669e-17,-2.921277e-19,1.700671e-17,...,3.874700e-18,-5.139569e-17,1.648313e-16,2.522157e-17,8.920840e-17,4.714523e-17,9.462153e-17,1.554883e-17,1.060731e-16,5.762924e-17
3,AL627309.1,-1.253259e-17,-4.773926e-17,-3.142607e-17,3.777894e-17,2.243308e-16,1.101191e-16,-5.938131e-17,1.150464e-16,-4.028997e-17,...,-8.959532e-17,4.638255e-18,-1.371361e-17,2.580507e-18,2.923144e-17,-2.168580e-17,5.690917e-17,-5.808644e-17,7.216122e-17,9.762675e-17
4,AL627309.3,-3.375304e-17,4.485008e-17,-7.486719e-17,-3.045501e-17,-5.758345e-17,-6.205900e-17,-2.527925e-17,2.098620e-17,5.838512e-17,...,-5.034477e-17,-8.140597e-17,-5.604782e-18,-3.805141e-17,4.402742e-18,-8.618553e-17,-9.480672e-17,-3.498366e-17,-2.389776e-17,-2.290068e-17


### Test of Covariance Matrices

In [ ]:
# Compute covariance matrices for each group
pc_cols = [col for col in scores_df.columns if col.startswith('PC')]
df_pc = scores_df[pc_cols + ['age']].copy()

# Calculate covariance matrices for each group
groups = df_pc['age'].unique()
cov_matrices = {}

for g in groups:
    data_group = df_pc[df_pc['age'] == g][pc_cols]
    cov_matrices[g] = np.cov(data_group, rowvar=False)
    print(f"Covariance matrix for group {g}:")
    print(cov_matrices[g], "\n")

# Box's M test for equality of covariance matrices
data_for_test = df_pc.copy()
data_for_test.rename(columns={'age': 'group'}, inplace=True)

X = data_for_test[pc_cols]
groups = data_for_test['group']
df_test = data_for_test[pc_cols + ['group']]

# Box's M test
box_m_results = pg.box_m(data=df_test, group='group', dvs=pc_cols)

print("Box's M test results:")
print(box_m_results)

# Extract the p-value
p_value = box_m_results['pval'].values[0]
print(f"\nBox's test p-value: {p_value:.2e}")

# Print the final
if p_value > 0.05:
    print("Covariance matrices are equal across groups --> **LDA**.")
else:
    print("Covariance matrices differ across groups --> **QDA**.")


Covariance matrix for group Pediatric_AML:
[[ 1.77412734e+02 -5.63080140e+00  4.90231542e+00 -3.16863257e+00
   8.34240791e+00 -8.58563372e+00 -5.99650832e+00  4.52503508e+00
  -4.21356188e+00  6.53066358e+00  3.42531905e+00 -7.25385896e-01
  -9.01314176e-01 -8.63837689e-01 -4.64236387e-01  1.02977024e-01
   1.35733212e-01 -2.63741521e+00 -1.14179408e+00 -9.41820385e-01
  -1.57922765e+00 -2.09000978e+00  5.72114451e-01 -1.74491003e+00
   1.11733952e+00  1.14453091e+00  2.63785206e+00  5.09865765e-02
  -6.30636797e-01 -1.40067788e+00]
 [-5.63080140e+00  4.00453866e+01 -4.37721099e-01 -3.96823142e+00
  -7.46837805e-01  2.42670467e+00 -3.00758024e-01  1.09143614e+00
  -1.74024212e+00 -4.30591192e+00 -1.02632125e+00  3.12060212e+00
   3.21184198e+00 -1.98904426e+00  2.56793326e+00 -5.09581349e-01
  -1.12889674e+00  1.16793543e+00 -1.19335223e+00  3.87668951e-01
   3.54384789e-01  6.87461792e-01  3.63713907e-01 -2.36077893e-01
   2.33957921e+00  3.20780590e-01  2.37888483e+00 -3.96759437e-0

/Users/eli/Library/Python/3.9/lib/python/site-packages/pingouin/multivariate.py:386: RuntimeWarning: divide by zero encountered in log
  u = -2 * (1 - c) * np.log(M)


## QDA

Since the covariance matrices differ across groups, we decided to use QDA instead of LDA

In [ ]:
# Perform QDA

X = scores_df.select_dtypes(include=[np.number])
y = scores_df["age"]

if not isinstance(X, (np.ndarray, pd.DataFrame)):
    X = X.toarray()

# Divide the dataset in train set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

# Compute the accuracy
y_pred_test = qda.predict(X_test)
acc = accuracy_score(y_test, y_pred_test)
print(f"Accuracy QDA: {acc:.4f}")

y_pred_all = qda.predict(X)
scores_df['cell_patient'] = scores_df['cell_type'] + "_" + scores_df['patient_id'].astype(str)

# Build a new dataframe with cells, age and the predicted label
result_df = pd.DataFrame({
    'cell_patient': scores_df['cell_patient'],
    'age': scores_df['age'],
    'predicted_label': y_pred_all
})

print(result_df.head())

Accuracy QDA: 0.9080
            cell_patient            age predicted_label
0     B cell_SRR16208932  Pediatric_AML   Pediatric_AML
1     B_prog_SRR16208932  Pediatric_AML   Pediatric_AML
2        CLP_SRR16208932  Pediatric_AML   Pediatric_AML
3  Early GMP_SRR16208932  Pediatric_AML   Pediatric_AML
4  Erythroid_SRR16208932  Pediatric_AML   Pediatric_AML


In [ ]:
# Predict the label by applying different weights according to cell type

df = result_df
df['patient_id'] = df['cell_patient'].str.extract(r'_(SRR\d+)$')
df['cell_type'] = df['cell_patient'].str.extract(r'^(.+?)_SRR')

# Define a dictionary of weights
cell_weights = {
    'B cell': 1, 'B_prog': 1, 'CLP': 1, 'Dendritic cell': 1, 'Early GMP': 3,
    'Erythroid': 1, 'GMP': 3, 'HSC': 1, 'LMPP': 3, 'MEP_prog': 3,
    'MPP': 3, 'Monocyte': 2, 'NK cell': 1, 'Pre-cDC': 2, 'Pre-pDC': 2,
    'ProMono': 3, 'Progenitor_Cycling': 1, 'T cell': 1, 'pDC': 1
}

df['weight'] = df['cell_type'].map(cell_weights)

# Group by patient_id and predicted_label, summing the weights
grouped = df.groupby(['patient_id', 'predicted_label'])['weight'].sum().reset_index()

# Select the predicted_label with the highest total weight for each patient
final_predictions = grouped.loc[grouped.groupby('patient_id')['weight'].idxmax()]

# Retrieve the true label for each patient
true_labels = df.groupby('patient_id')['age'].first().reset_index()

# Merge predicted and true labels
final_results = pd.merge(final_predictions, true_labels, on='patient_id')

final_results = final_results.rename(columns={
    'predicted_label': 'predicted',
    'age': 'true_label',
    'weight': 'total_weight'
})

print(final_results)


     patient_id      predicted  total_weight     true_label
0   SRR16208921      Adult_AML            34      Adult_AML
1   SRR16208922      Adult_AML            33      Adult_AML
2   SRR16208923      Adult_AML            33      Adult_AML
3   SRR16208924          CNTRL            32          CNTRL
4   SRR16208925          CNTRL            33          CNTRL
5   SRR16208926          CNTRL            33          CNTRL
6   SRR16208927          CNTRL            33          CNTRL
7   SRR16208928      Adult_AML            34      Adult_AML
8   SRR16208929      Adult_AML            34      Adult_AML
9   SRR16208930  Pediatric_AML            34  Pediatric_AML
10  SRR16208931  Pediatric_AML            34  Pediatric_AML
11  SRR16208932  Pediatric_AML            33  Pediatric_AML
12  SRR16208937  Pediatric_AML            34  Pediatric_AML
13  SRR16208938  Pediatric_AML            34  Pediatric_AML
14  SRR16208939  Pediatric_AML            34  Pediatric_AML
15  SRR16208940  Pediatric_AML          

## SUPPORT VECTOR MACHINE

In [ ]:

feature_columns = [col for col in scores_df.columns if col.startswith('PC')]
X = scores_df[feature_columns].values
y = scores_df['cell_type'].values

# Split in test and training
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Evaluation
y_pred = svm_model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))



Accuracy: 0.8625954198473282
Classification Report:
                    precision    recall  f1-score   support

            B cell       1.00      1.00      1.00         7
            B_prog       1.00      1.00      1.00         7
               CLP       0.67      0.57      0.62         7
    Dendritic cell       1.00      1.00      1.00         6
         Early GMP       0.67      0.57      0.62         7
         Erythroid       1.00      1.00      1.00         7
               GMP       1.00      0.86      0.92         7
               HSC       0.67      0.57      0.62         7
              LMPP       0.80      0.57      0.67         7
          MEP_prog       0.88      1.00      0.93         7
               MPP       0.50      0.71      0.59         7
          Monocyte       1.00      1.00      1.00         7
           NK cell       1.00      1.00      1.00         7
           Pre-cDC       1.00      0.86      0.92         7
           Pre-pDC       0.83      0.71      0.

## Determine the genes with the highest discriminative power

In [ ]:
loadings_df.head

<bound method NDFrame.head of              genes           PC1           PC2           PC3           PC4  \
0      MIR1302-2HG -4.112838e-18 -5.568293e-17 -5.082637e-17 -1.407835e-16   
1          FAM138A -6.331741e-17  4.336809e-17 -9.020562e-17 -1.597572e-16   
2            OR4F5 -4.465181e-17  6.150701e-17 -7.694165e-17 -4.027674e-17   
3       AL627309.1 -1.253259e-17 -4.773926e-17 -3.142607e-17  3.777894e-17   
4       AL627309.3 -3.375304e-17  4.485008e-17 -7.486719e-17 -3.045501e-17   
...            ...           ...           ...           ...           ...   
36596   AC141272.1 -0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
36597   AC023491.2 -0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
36598   AC007325.1 -0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
36599   AC007325.4 -0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
36600   AC007325.2 -0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   

                PC5           PC6

In [ ]:
pc_columns = [col for col in scores_df.columns if col.startswith('PC')]
df_pc = scores_df[pc_columns]

# Compute the covariance matrix and its eigenvalues
cov_matrix = np.cov(df_pc.T)
eigenvalues = np.linalg.eigvals(cov_matrix)

# Print the eigenvalues
print("Eigenvalues of the covariance matrix:")
for i, val in enumerate(eigenvalues, start=1):
    print(f"{i}. {val:.4f}")

# Calculate the proportion of variance explained by each component
proportions = eigenvalues / eigenvalues.sum()


Eigenvalues of the covariance matrix:
1. 121.2080
2. 33.3060
3. 28.7859
4. 23.1760
5. 15.6986
6. 12.4847
7. 10.5079
8. 8.3967
9. 7.5110
10. 7.1303
11. 6.0458
12. 4.8045
13. 4.3134
14. 4.1763
15. 3.8696
16. 3.2113
17. 3.0821
18. 2.8516
19. 2.5636
20. 2.5007
21. 1.6279
22. 1.6865
23. 2.3384
24. 1.7799
25. 1.8563
26. 1.9084
27. 2.2248
28. 2.1172
29. 2.0523
30. 2.0744


In [ ]:
# Choose the total number of selected genes for each PC
GENES_PER_PC = 9
NUM_PCS = 30

variable_genes = []

# Select top genes from each PC
for i in range(1, NUM_PCS + 1):
    pc = f'PC{i}'

    # Sort genes by descending loading on the current PC
    top_genes = loadings_df.nlargest(GENES_PER_PC, pc)['genes'].tolist()
    variable_genes.extend(top_genes)

# Remove duplicates while preserving original order
variable_genes = list(dict.fromkeys(variable_genes))

print("First 10 selected genes:", variable_genes[:10])
print("Total number of selected genes (after removing duplicates):", len(variable_genes))


First 10 selected genes: ['CHST11', 'ETV6', 'LRMDA', 'ATP8B4', 'MED13L', 'GAB2', 'MAML3', 'SSBP2', 'ANKRD28', 'MALAT1']
Total number of selected genes (after removing duplicates): 179


In [ ]:
print("Selected genes:")
print(variable_genes)

cols_base = ['patient_id', 'cell_type', 'age'] + variable_genes

# Create the filtered DataFrame
filtered_rna = grouped_rna.loc[:, cols_base]

# Informative output
print(filtered_rna.head())
print(f"Shape of the filtered DataFrame: {filtered_rna.shape}")


Selected genes:
['CHST11', 'ETV6', 'LRMDA', 'ATP8B4', 'MED13L', 'GAB2', 'MAML3', 'SSBP2', 'ANKRD28', 'MALAT1', 'RPL10', 'MT-CO3', 'RPL13', 'MT-ATP6', 'MT-CO2', 'RPS12', 'RPS18', 'MT-CO1', 'LYZ', 'S100A9', 'S100A8', 'S100A6', 'SRGN', 'CST3', 'S100A4', 'VCAN', 'FTL', 'RPLP1', 'SNHG29', 'GAPDH', 'GSTP1', 'RPS8', 'RPS24', 'AFF3', 'RABGAP1L', 'CALM1', 'BACH2', 'EML4', 'IGKC', 'LINC01619', 'RPS27', 'HDAC9', 'HLA-DRA', 'CD74', 'HLA-DRB1', 'HLA-DPB1', 'FCHSD2', 'TCF4', 'HLA-DPA1', 'UBE2E2', 'HBB', 'HBA1', 'HBA2', 'HBD', 'AHSP', 'PRDX2', 'GYPC', 'FTH1', 'BLVRB', 'TMEM131L', 'SLC25A37', 'FOXP1', 'EZR', 'CAMK1D', 'EBF1', 'BANK1', 'BTG1', 'CDK14', 'SIPA1L1', 'TRPS1', 'AUTS2', 'ZEB1', 'ATP2B1', 'MT-ND3', 'MSI2', 'XIST', 'MT-ND2', 'JUN', 'NEGR1', 'KLF6', 'IER2', 'MT-ND5', 'PLXDC2', 'TOX', 'NRIP1', 'JARID2', 'ANGPT1', 'MED12L', 'S100A10', 'SOX4', 'PDE4D', 'AZU1', 'SAMHD1', 'MPO', 'PELI2', 'ZFP36L2', 'CHSY1', 'STMN1', 'HMGN2', 'ZEB2', 'HMGB2', 'HMGB1', 'ACSM3', 'B2M', 'IFITM2', 'GNLY', 'DPYD', 'FOS', 

In [ ]:
# Perform QDA using only the 179 selected genes
X = filtered_rna.select_dtypes(include=[np.number])
y = filtered_rna["age"]

if not isinstance(X, (np.ndarray, pd.DataFrame)):
    X = X.toarray()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# QDA model
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

# Predict on the test set and evaluate
y_pred_test = qda.predict(X_test)
acc = accuracy_score(y_test, y_pred_test)
print(f"QDA Accuracy: {acc:.4f}")

# Predict on the entire dataset
y_pred_all = qda.predict(X)

# Build a new DataFrame
filtered_rna['cell_patient'] = filtered_rna['cell_type'] + "_" + filtered_rna['patient_id'].astype(str)

gene_result_df = pd.DataFrame({
    'cell_patient': filtered_rna['cell_patient'],
    'age': filtered_rna['age'],
    'predicted_label': y_pred_all
})

# Identify mismatches between true and predicted labels
mismatched_rows = gene_result_df[gene_result_df['age'] != gene_result_df['predicted_label']]
print(mismatched_rows)
print(f"Number of misclassifications: {len(mismatched_rows)}")


QDA Accuracy: 0.6782
                       cell_patient            age predicted_label
0                B cell_SRR16208932  Pediatric_AML           CNTRL
15   Progenitor_Cycling_SRR16208932  Pediatric_AML       Adult_AML
30              NK cell_SRR16208931  Pediatric_AML           CNTRL
33              ProMono_SRR16208931  Pediatric_AML           CNTRL
42            Erythroid_SRR16208930  Pediatric_AML       Adult_AML
56               B cell_SRR16208925          CNTRL   Pediatric_AML
57               B_prog_SRR16208925          CNTRL   Pediatric_AML
75               B cell_SRR16208927          CNTRL   Pediatric_AML
94               B cell_SRR16208926          CNTRL   Pediatric_AML
113              B cell_SRR16208924          CNTRL       Adult_AML
154           Early GMP_SRR16208938  Pediatric_AML       Adult_AML
155           Erythroid_SRR16208938  Pediatric_AML           CNTRL
180            Monocyte_SRR16208942  Pediatric_AML       Adult_AML
249           Early GMP_SRR16208948  Pedi

/Users/eli/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/Users/eli/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/Users/eli/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 2 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(


In [ ]:
# Predict the label by applying different weights according to cell type
df = gene_result_df
df['patient_id'] = df['cell_patient'].str.extract(r'_(SRR\d+)$')
df['cell_type'] = df['cell_patient'].str.extract(r'^(.+?)_SRR')

# Cell type weights dictionary
cell_weights = {
    'B cell': 1, 'B_prog': 1, 'CLP': 1, 'Dendritic cell': 1, 'Early GMP': 3,
    'Erythroid': 1, 'GMP': 3, 'HSC': 1, 'LMPP': 3, 'MEP_prog': 3,
    'MPP': 3, 'Monocyte': 2, 'NK cell': 1, 'Pre-cDC': 2, 'Pre-pDC': 2,
    'ProMono': 3, 'Progenitor_Cycling': 1, 'T cell': 1, 'pDC': 1
}

df['weight'] = df['cell_type'].map(cell_weights)

# Group by patient_id and predicted_label, sum weights
grouped = df.groupby(['patient_id', 'predicted_label'])['weight'].sum().reset_index()

# Select predicted_label with the highest total weight per patient
final_predictions = grouped.loc[grouped.groupby('patient_id')['weight'].idxmax()]

# Get true label for each patient
true_labels = df.groupby('patient_id')['age'].first().reset_index()

# Merge predicted and true labels
final_results = pd.merge(final_predictions, true_labels, on='patient_id')

final_results = final_results.rename(columns={
    'predicted_label': 'predicted',
    'age': 'true_label',
    'weight': 'total_weight'
})

# Show final result
print(final_results)


     patient_id      predicted  total_weight     true_label
0   SRR16208921      Adult_AML            27      Adult_AML
1   SRR16208922      Adult_AML            29      Adult_AML
2   SRR16208923      Adult_AML            33      Adult_AML
3   SRR16208924          CNTRL            32          CNTRL
4   SRR16208925          CNTRL            32          CNTRL
5   SRR16208926          CNTRL            33          CNTRL
6   SRR16208927          CNTRL            33          CNTRL
7   SRR16208928      Adult_AML            34      Adult_AML
8   SRR16208929      Adult_AML            34      Adult_AML
9   SRR16208930  Pediatric_AML            33  Pediatric_AML
10  SRR16208931  Pediatric_AML            30  Pediatric_AML
11  SRR16208932  Pediatric_AML            31  Pediatric_AML
12  SRR16208937  Pediatric_AML            34  Pediatric_AML
13  SRR16208938  Pediatric_AML            30  Pediatric_AML
14  SRR16208939  Pediatric_AML            29  Pediatric_AML
15  SRR16208940  Pediatric_AML          